# 清理资源

本笔记本演示了如何清理在上一组笔记本中创建的所有资源。

In [ ]:
import boto3
import json

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r

请注意，这些步骤取决于每个步骤都已完全完成，然后另一个步骤才能起作用。 如果收到错误，请等待一两分钟，然后重试。

## 清理活动（Campaigns）

首先清理活动（Campaigns）。 您需要先删除Campaigns，然后才能删除它们所基于的Solutions版本。

以下代码将删除您创建的Campaigns。

In [ ]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

## 清理Event Trackers

接下来，清理所有Event Trackers。 以下代码将删除您的Event Trackers。

In [ ]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

## 清理Filters

接下来，清理所有Filters。 下面的代码遍历您创建的所有Filters并删除它们。 请注意，如果您收到关于Campaigns的错误，没关系，请等待2分钟，然后再次尝试下面的单元格。

注意：在删除Campaigns之前，您将无法删除Filters，这可能需要几分钟的时间。

In [ ]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## 清理Solutions

接下来，清理Solutions。 下面的代码将删除创建的Solutions。

In [ ]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

## 清理Datasets

接下来，清理Datasets。 下面的代码将删除items datasets和interactions datasets。

In [ ]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

## 清理Schemas

接下来，清理Schemas。 您创建了 interaction schema 和 item schema，现在将其删除。

In [ ]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

## 清理Dataset Groups

最后，清理Dataset Groups。

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

## 清理S3存储桶和IAM Role

首先删除IAM Role，然后清空存储桶，最后删除存储桶。

In [ ]:
iam = boto3.client('iam')

标识要删除的IAM Role的名称。

您无法删除仍附加有策略的IAM Role。 因此，在确定了相关IAM Role后，让我们列出该IAM Role的附加策略。

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

您需要使用以下代码分离以上结果中的策略。 对每个附加策略重复此操作。

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

最后，您应该能够删除IAM Role。

In [ ]:
iam.delete_role(
    RoleName = role_name
)

要删除S3存储桶，首先需要将其清空。 删除S3存储桶的最简单方法是，在亚马逊云科技控制台中导航到S3，删除存储桶中的对象，然后删除S3存储桶本身。